In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Notes
### Import the notes file

In [ ]:
df = pd.read_csv("./notes_total.csv", names=["note", "count", "title"], sep=";")
df.sort_values("count", ascending=False)

### Create the plots

In [ ]:
grouped = df.groupby("note").sum().sort_values("count", ascending=False).reset_index()
natural = grouped["note"].isin(['A', 'B', 'C', 'D', 'E', 'F', 'G'])
sharp = grouped["note"].isin(['A#', 'B#', 'C#', 'D#', 'E#', 'F#', 'G#'])
flat = grouped["note"].isin(['Ab', 'Bb', 'Cb', 'Db', 'Eb', 'Fb', 'Gb'])
rest = ~natural & ~sharp & ~flat


plt.figure(figsize=(15,40))

plt.subplot(411)
plt.bar(x=grouped[natural]["note"], height = grouped[natural]["count"])
plt.title('Natural Notes')

plt.subplot(412)
plt.bar(x=grouped[sharp]["note"], height = grouped[sharp]["count"])
plt.title('Sharp Notes')

plt.subplot(413)
plt.bar(x=grouped[flat]["note"], height = grouped[flat]["count"])
plt.title('Flat Notes')

plt.subplot(414)
plt.bar(x=grouped[rest]["note"], height = grouped[rest]["count"])
plt.title('Other Notes')

plt.show()

# 2. Chords
### Import the chords file

In [ ]:
df = pd.read_csv("./chords_total.csv", names=["comb", "count", "title"], sep=";")
df.sort_values("count", ascending=False)

### Group by specific chord

In [ ]:
gbcomb = df.groupby("comb").sum()["count"].sort_values(ascending=False).reset_index()
gbcomb

### Group by n-chord

Converting chords to their n-note set, assuming inversional equivalency.

In [ ]:
import itertools

# Notes dictionary for 2 octaves, so notes can appear in any permutation
notes_dic = {'C': [1, 13], 'C#': [2, 14], 'D': [3, 15], 'D#': [4, 16], 'E': [5, 17], 'F': [6, 18], 
             'F#': [7, 19], 'G': [8, 20], 'G#': [9, 21], 'A': [10, 22], 'A#': [11, 23], 
             'B': [12, 24], 'Db': [2, 14], 'Eb': [4, 16], 'Gb': [7, 19], 
             'Ab': [9, 21], 'Bb': [11, 23], 'E#': [6, 18], 'B#': [1, 13], 'Fb': [5, 17], 'Cb': [12, 24],
             'C##': [3, 15], 'F##': [8, 18], "G##": [10, 22], "Bbb": [10, 22]}

# Dictionary with traditional names of some n-chords
chord_dict = {'0': 'Unison or Single Note', '037': 'Minor Chord', '047': 'Major Chord', 
              '036': 'Dim Chord', '048': 'Aug Chord', '0368': 'Dom 7 Chord', '027': 'Sus Chord', 
              '026': "It6 Chord", '025': "mm Chord", '0247': 'Mu Chord', '0358': 'Min7 Chord', 
              '035': 'Blues Chord', '0158': 'Major 7 Chord', '0258': 'Half-dim 7 Chord',
              '01': 'Min Second Interval', '02': 'Maj Second Interval', 
              '03': 'Min Third Interval', '04': 'Maj Third Interval', '05': 'Perfect Fourth Interval', 
              '06': 'Aug Fourth Interval', '07': 'Perfect Fifth Interval'}

def num_chords(lst):
    
    # Convert to numbers. Skip null values
    ls = []
    for note in lst:
        try:
            ls.append(notes_dic[note])
        except:
            continue
    
    # Get all possible combinations. This gives all possible positions and inversions of a n-chord.
    comb_list = list(itertools.product(*ls))
    
    # We need to obtain the more compact form of an n-chord, i.e. the order of notes that minimizes 
    # the sum of the distances of the notes.
    if len(ls) > 1:

        comb_array = np.array(comb_list)
        diff = np.max(comb_array, axis=1) - np.min(comb_array, axis=1)
        min_idxs = np.where(diff == diff.min())
        min_arrays = comb_array[min_idxs]
        min_arrays = np.sort(min_arrays, axis=1)

        for i, min_array in enumerate(min_arrays):
            min_arrays[i] = min_array - min_array[0]
        fin_array = np.sort(min_arrays, axis=0)[0]

        return tuple(fin_array)
    else:
        return tuple([0])


def tryconvert(x):
    try:
        chord = chord_dict[x]
    except:
        chord = x
    
    return chord


print(num_chords(['C', 'F', 'G']))

### Applying the conversion

In [ ]:
gbcomb["comb_list"] = gbcomb["comb"].apply(lambda x: x.split(" "))
gbcomb["num_chords"] = gbcomb["comb_list"].apply(num_chords)
gbcomb["text_num_chord"] = gbcomb["num_chords"].apply(lambda x: ''.join([str(ch) for ch in x]))
gbcomb["chord"] = gbcomb["text_num_chord"].apply(tryconvert)

grouped = gbcomb.groupby("chord").sum()["count"].sort_values(ascending=False).reset_index()
grouped

### Creating the plot

In [ ]:
grouped = grouped.head(25).sort_values("count")

plt.figure(figsize=(20,80))
plt.subplot(411)

plt.barh(y=grouped["chord"], width = grouped["count"])
plt.yticks(size=22)
plt.title('Chord Frequency', size=26)

plt.show()

### Fit a curve to the data

Fit a Hurwitz zeta function to the data, 

\begin{equation*}
f(k,q,s) = \frac{C}{(k+q)^s}
\end{equation*}

according to Zipf–Mandelbrot law

In [ ]:
from scipy.optimize import curve_fit

def func(x, a, b, c):
    return a /(x + b)**c


xdata = np.arange(1, gbcomb.shape[0]+1)
ydata = gbcomb["count"]

popt, pcov = curve_fit(func, xdata, ydata)
gbcomb["fit"] = func(xdata, *popt)

### Create the plot

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
line1 = ax.scatter(xdata, ydata, label='Frequency', c="tomato", linewidth=1.5)
ax.tick_params(direction="in", which="both")
ax.set_yscale('log')
ax.set_xscale('log')
line2 = ax.plot(xdata, gbcomb["fit"], dashes=[3, 3, 10, 3], label='Fit', c="black", linewidth=1.5)

ax.legend()
plt.show()

# 3. Chord Progression
### Import the files and merge them into one dataframe

In [ ]:
df = pd.read_csv("./strings_total.csv", names=["title", "string"], sep=";")
df["num"] = df.title.apply(lambda x: x.split("_")[0][3:])

keys = pd.read_csv("./keys.csv", names=["num", "key"], sep=",")
df = df.merge(keys, on="num")
documents = df.string

### Get chord progression from documents as n-grams

In [ ]:
from nltk import ngrams
from nltk.probability import FreqDist

df["chors_list"] = df.string.apply(lambda x: x.split(" "))

# n can be adjusted to values other than 2
df["bigrams"] = df.chors_list.apply(lambda x: list(ngrams(x, 2)))

biglist = df.bigrams.sum()
biglist

### Filtering

Include only n-grams (chord progressions) having atleast one trichord or higher

In [ ]:
filtered = []
for prog in biglist:
    len_list = []
    for chord in prog:
        note_list = chord.strip().split("-")
        fin_len = len(note_list)
        len_list.append(fin_len)
    len_arr = np.array(len_list)
    cond = np.all(len_arr > 0) and np.any(len_arr > 2)
    if cond:
        filtered.append(prog)

filtered

### Chord conversion
Convert chords to note sets and get the most used chord progressions

In [ ]:
def convert_chord(string):
    # Convert chords using num_chords function from section 2
    ls = string.split("-")
    tup = num_chords(ls)
    
    return tup


conv_prog_list = []
for tup in filtered:
    prog_list = []
    for string in tup:
        ls = convert_chord(string)
        prog_list.append(ls)
    conv_prog_list.append(tuple(prog_list))

freq_prog = FreqDist(conv_prog_list).most_common()
freq_prog

### Convert to traditional chord names

In [ ]:
def trad_conv_chord(x):
    chord_list = []
    for num_chord in x:
        chord = tryconvert(num_chord)
        chord_list.append(chord)
    return chord_list

df_prog = pd.DataFrame(freq_prog, columns=["Prog", "Freq"])
df_prog["num_chord"] = df_prog["Prog"].apply(lambda x: [''.join(str(ch) for ch in tup) for tup in x])

df_prog["chord_name"] = df_prog["num_chord"].apply(trad_conv_chord)
df_prog["chrd_prog"] = df_prog["chord_name"].apply(" -> ".join)

df_prog.head(60)

### Creating the plot

In [ ]:
grouped = df_prog.head(25).sort_values("Freq")

plt.figure(figsize=(20,80))
plt.subplot(411)

plt.barh(y=grouped["chrd_prog"], width = grouped["Freq"])
plt.yticks(size=22)
plt.title('Chord Frequency', size=26)

plt.show()

# 4. Clusters
### Train several models to determine the optimal k for k-means clustering

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score


vectorizer = TfidfVectorizer(sublinear_tf=True, token_pattern=r"(?u)\S\S+", lowercase=False)
X = vectorizer.fit_transform(documents)

sil = []
distances = []
K = range(2,20)
for k in K:
    km = KMeans(n_clusters=k, init='k-means++')
    km = km.fit(X)
    labels = km.labels_
    distances.append(km.inertia_)
    sil.append(silhouette_score(X, labels, metric = 'euclidean'))


### Use elbow and silhouette methods to determine optimal k

Looking at the plots, there is no clear elbow, but we can see the slope slightly changing at $k=7$. The silhouette plot further confirms this

In [ ]:
plt.figure(figsize=(16,6))

plt.subplot(121)
plt.plot(K, distances, 'bo-')
plt.tick_params(direction="in")
plt.xlabel('k')
plt.ylabel('Sum of square distances')
plt.title('Elbow Method')

plt.subplot(122)
plt.plot(K, sil, 'bo-')
plt.tick_params(direction="in")
plt.xlabel('k')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Method')
plt.show()

### Get clusters for $k=7$

In [ ]:
true_k = 7
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=300, n_init=20)
model.fit(X)


prediction = model.predict(X)
df["cluster"] = prediction
print(df["cluster"].value_counts())

### Get the top terms of each cluster

In [ ]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i)
    print(df[df["cluster"]==i]["key"].value_counts())
    for ind in order_centroids[i, :15]:
        print(' %s' % terms[ind])